In [21]:
import torch
torch.__version__

'2.9.1+cpu'

In [22]:
#!pip install scikit-learn
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [32]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path

train_dir = Path("../data/split/train")
val_dir   = Path("../data/split/val")


In [33]:
train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

val_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])


In [34]:
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(val_dir,   transform=val_transforms)


In [35]:
print("Classes:", train_dataset.classes)
print("Num classes:", len(train_dataset.classes))
img, label = train_dataset[0]
print(img.shape, label)
print(" number of images " , len(train_dataset))


Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Num classes: 6
torch.Size([3, 64, 64]) 0
 number of images  1767


In [36]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False)


In [37]:
import torch
import torch.nn as nn
class SimpleCNN(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()

        # Convolutional backbone
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),     # 64 -> 32

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),     # 32 -> 16
        )

        # Flatten size after convs: 32 * 16 * 16
        self.embedding_dim = 32 * 16 * 16

        # Embedding head (this is what we’ll use later)
        self.embedding_head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.embedding_dim, 128),
            nn.ReLU(),
        )

        # Classification head (uses the 128-dim embedding)
        self.classifier_head = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128, num_classes),
        )

    def forward_features(self, x: torch.Tensor) -> torch.Tensor:
        """
        Returns the 128-dim embedding vector.
        This is what the EmbeddingClient will call.
        """
        x = self.features(x)
        x = self.embedding_head(x)
        return x  # shape: (batch, 128)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.forward_features(x)
        x = self.classifier_head(x)
        return x  # logits


In [38]:
num_classes = len(train_dataset.classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)


In [39]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4,
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.5,
    patience=3,
)

In [40]:
import time

num_epochs = 16
total_start = time.time()

for epoch in range(num_epochs):
    epoch_start = time.time()

    # ----- TRAIN -----
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

       
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_acc = correct_train / total_train

    # ----- VALIDATION -----
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    scheduler.step(avg_val_loss)

    val_acc = correct_val / total_val

    epoch_time = time.time() - epoch_start

    print(
        f"Epoch {epoch+1}/{num_epochs} "
        f"- Train Loss: {avg_train_loss:.4f} - Train Acc: {train_acc:.4f} "
        f"- Val Loss: {avg_val_loss:.4f} - Val Acc: {val_acc:.4f} "
        f"- Time: {epoch_time:.2f} sec"
    )

total_time = time.time() - total_start
print(f"\nTotal training time: {total_time:.2f} sec")


Epoch 1/16 - Train Loss: 1.6401 - Train Acc: 0.3294 - Val Loss: 1.3811 - Val Acc: 0.4828 - Time: 17.67 sec
Epoch 2/16 - Train Loss: 1.3773 - Train Acc: 0.4743 - Val Loss: 1.2287 - Val Acc: 0.5358 - Time: 15.55 sec
Epoch 3/16 - Train Loss: 1.1892 - Train Acc: 0.5563 - Val Loss: 1.1348 - Val Acc: 0.5756 - Time: 19.61 sec
Epoch 4/16 - Train Loss: 1.0834 - Train Acc: 0.5937 - Val Loss: 1.0477 - Val Acc: 0.5942 - Time: 25.61 sec
Epoch 5/16 - Train Loss: 0.9752 - Train Acc: 0.6378 - Val Loss: 0.9933 - Val Acc: 0.6127 - Time: 24.19 sec
Epoch 6/16 - Train Loss: 0.9285 - Train Acc: 0.6672 - Val Loss: 0.9538 - Val Acc: 0.6472 - Time: 36.69 sec
Epoch 7/16 - Train Loss: 0.8274 - Train Acc: 0.6882 - Val Loss: 0.9002 - Val Acc: 0.6631 - Time: 48.63 sec
Epoch 8/16 - Train Loss: 0.7797 - Train Acc: 0.7085 - Val Loss: 0.9255 - Val Acc: 0.6844 - Time: 36.25 sec
Epoch 9/16 - Train Loss: 0.7313 - Train Acc: 0.7323 - Val Loss: 0.9035 - Val Acc: 0.6870 - Time: 28.12 sec
Epoch 10/16 - Train Loss: 0.6479 - Tr

In [44]:
from pathlib import Path
import json
import torch
import os

print("Notebook running from:", os.getcwd())

# go one level up from notebooks/ → project root
save_dir = Path("../models/classifier")
save_dir.mkdir(parents=True, exist_ok=True)

# 2) Save model weights
model_path = save_dir / "simple_cnn.pth"
torch.save(model.state_dict(), model_path)
print(f"Saved model weights to: {model_path}")

# 3) Save class names
class_names = train_dataset.classes
classes_path = save_dir / "classes.json"
with open(classes_path, "w") as f:
    json.dump(class_names, f)

print(f"Saved class names to: {classes_path}")
print("Classes:", class_names)


Notebook running from: C:\Users\sadek\OneDrive\Desktop\DSAI4101-project\notebooks
Saved model weights to: ..\models\classifier\simple_cnn.pth
Saved class names to: ..\models\classifier\classes.json
Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
